In [ ]:
First off install dependencies. This assumes that the notebook is run in the correct virtualenv.

In [ ]:
!pip install pandas numpy seaborn gpy sklearn plotly

In [7]:
# Rename and repackage data


import pandas as pd
path = '../data/203_train.p'
data = pd.read_pickle(path)

# n_trajs = list(data.traj.unique())
# n_trajs_to_use = 5 # len(n_trajs)

data = data.rename(columns = {
    'latitude': 'lat',
    'longitude': 'lon',
    'journey_number': 'traj',
    'segment_number': 'seg', 
    'speed': 'speed',
    'event': 'event',
    'timestamp': 'timestamp'
})
data.to_pickle('../data/train3.pkl')
data.shape

(2769463, 12)

In [ ]:
from IPython.display import display
import numpy as np
import GPy

X = np.random.uniform(-3.,3.,(20,1))
Y = np.sin(X) + np.random.randn(20,1)*0.05
kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
m = GPy.models.GPRegression(X,Y,kernel)
print(X.shape, Y.shape)
print(m)
# GPy.plotting.change_plotting_library('plotly')
fig = m.plot()
#GPy.plotting.show(fig, filename='basic_gp_regression_notebook')



In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
from IPython.display import display, clear_output

%matplotlib inline
sns.set(style="darkgrid")

trajs_pickle_path = '../data/train3.pkl'
data = pd.read_pickle(trajs_pickle_path)
n_trajs = list(data.traj.unique())
n_trajs_to_use = 5 # len(n_trajs)

np.random.seed(12345)
trajs_to_use = frozenset(np.random.choice(n_trajs, size=n_trajs_to_use, replace=False))
trajs = data[data.traj.transform(lambda j: j in trajs_to_use)]
traj_ixs = trajs.traj.unique()
trajs.head()


         Unnamed: 0                          timestamp                  event  \
2726800           0  2018-04-06T04:46:49.0000000+02:00    JourneyStartedEvent   
2726801           1  2018-04-06T04:46:50.0000000+02:00  ObservedPositionEvent   
2726802           2  2018-04-06T04:46:51.0000000+02:00  ObservedPositionEvent   
2726803           3  2018-04-06T04:46:52.0000000+02:00  ObservedPositionEvent   
2726804           4  2018-04-06T04:46:53.0000000+02:00  ObservedPositionEvent   

         vehicle_id  line        lon        lat   direction  speed station  \
2726800        5415   203  58.414219  15.571022   -1.000000   -1.0     NaN   
2726801        5415     0  58.414227  15.571007  144.699997    0.0     NaN   
2726802        5415     0  58.414230  15.570998  144.699997    0.0     NaN   
2726803        5415     0  58.414230  15.570992  144.699997    0.0     NaN   
2726804        5415     0  58.414230  15.570991  144.699997    0.0     NaN   

         traj  seg  
2726800  2875    1  
27

In [2]:
from model import learn_trajectory_model, plot_function

traj_one = trajs[trajs.traj == traj_ixs[0]]
m = learn_trajectory_model(traj_one, ['lat', 'lon'], ['lat', 'lon'])

Optimization restart 1/3, f = -5661.289530998933


Optimization restart 2/3, f = -5661.289531066791


Optimization restart 3/3, f = -5661.289531075537


Optimization restart 1/3, f = -859.9644967455217


Optimization restart 2/3, f = -859.9644967424852


Optimization restart 3/3, f = -859.9644967452803


 /home/sebca553/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning:Data with input dtype int64 were all converted to float64 by StandardScaler.
 /home/sebca553/Documents/msc/model/model.py:107: DataConversionWarning:Data with input dtype int64 were all converted to float64 by StandardScaler.


Optimization restart 1/3, f = -3432.530670830555


Optimization restart 2/3, f = -3432.530670860052


Optimization restart 3/3, f = -3798.618018935231
